In [ ]:
!pip install transformers datasets torch

In [ ]:
import torch
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [ ]:
# Load dataset (1% of the training set)
dataset = load_dataset('amazon_polarity', split='train[:1%]')

# Split dataset into train and test
split = dataset.train_test_split(test_size=0.1)
train_dataset = split['train']
test_dataset = split['test']


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['content'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=32)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=32)

# Set formats for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to=[]  # Disable all integrations including wandb
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()


In [ ]:
import matplotlib.pyplot as plt

metrics = trainer.state.log_history

# Extract train and eval loss with indices
train_loss = [x['loss'] for x in metrics if 'loss' in x]
eval_metrics = [(i, x['eval_loss']) for i, x in enumerate(metrics) if 'eval_loss' in x]

# Separate indices and values for eval loss
eval_steps, eval_loss = zip(*eval_metrics) if eval_metrics else ([], [])

steps = range(len(train_loss))

plt.plot(steps, train_loss, label="Train Loss")
if eval_metrics:
    plt.plot(eval_steps, eval_loss, label="Eval Loss")
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training and Evaluation Loss Over Steps')
plt.legend()
plt.show()


In [ ]:
# Select test samples - first 5 rows for instance
test_samples = test_dataset.select(range(5))  # Assuming test_dataset is available

# Run prediction on those samples using your trainer
preds = trainer.predict(test_samples)

# Extract predicted class labels from model output logits
predicted_labels = preds.predictions.argmax(axis=1)


In [ ]:
for i, sample in enumerate(test_samples):
    for key in ['title', 'content']:
        if key in sample:
            print(f"{key.capitalize()}: {sample[key]}")
    print(f"True Label: {sample['label']}, Predicted: {predicted_labels[i]}")
    print("---")


In [ ]:
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation results:", eval_results)


In [ ]:
model.save_pretrained('./fine_tuned_bert_amazon')
tokenizer.save_pretrained('./fine_tuned_bert_amazon')
